# Processing Metadata

**Methodology**:
1. Extract pertinent metadata feature (field management practises/ geographical locations/ Issues)
2. Calculate central lattitude/longtitude + Manage Issues columns
3. Fill in missing lat/lon values
4. Label encode and noramlise features (where appropriate)


In [1]:
import pandas as pd
print(pd.__version__)

2.2.3


In [ ]:
df_metadata = pd.read_csv('2_Training_Meta_Data_2014_2023.csv')
df_metadata.head(10)

,Year,Env,Experiment_Code,Treatment,City,Farm,Field,Trial_ID (Assigned by collaborator for internal reference),"Soil_Taxonomic_ID and horizon description, if known","Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)",...,Cardinal_Heading_Pass_1,Irrigated,Issue/comment_#1,Issue/comment_#2,Issue/comment_#3,Issue/comment_#4,Issue/comment_#5,Issue/comment_#6,Issue/comment_#7,Issue/comment_#8
0,2014,DEH1_2014,DEH1,NaN,Georgetown,Elbert N. & Ann V. Carvel Research & Education...,27AB,NaN,NaN,9079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,GAH1_2014,GAH1,NaN,Tifton,USDA - Bellflower experimental farm,18,NaN,NaN,8427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,IAH1a_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN
3,2014,IAH1b_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN
4,2014,IAH1c_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN
5,2014,IAH2_2014,IAH2,NaN,Carroll,NaN,NaN,NaN,NaN,9083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2014,IAH3_2014,IAH3,NaN,Keystone,NaN,NaN,NaN,NaN,9085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2014,IAH4_2014,IAH4,NaN,Crawfordsville,Southeast Research Farm,14,NaN,NaN,9082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2014,ILH1_2014,ILH1,NaN,Urbana,Maxwell Farms,MF500,NaN,NaN,8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014,INH1_2014,INH1,NaN,West Lafayette,Purdue ACRE,97/98,NaN,NaN,8657,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
len(df_metadata)
# metadata info available for 272 year-locations

272

In [ ]:
df_metadata.columns

Index(['Year', 'Env', 'Experiment_Code', 'Treatment', 'City', 'Farm', 'Field',
       'Trial_ID (Assigned by collaborator for internal reference)',
       'Soil_Taxonomic_ID and horizon description, if known',
       'Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)',
       'Weather_Station_Latitude (in decimal numbers NOT DMS)',
       'Weather_Station_Longitude (in decimal numbers NOT DMS)',
       'Date_weather_station_placed', 'Date_weather_station_removed',
       'Previous_Crop', 'Pre-plant_tillage_method(s)',
       'In-season_tillage_method(s)',
       'Type_of_planter (fluted cone; belt cone; air planter)',
       'System_Determining_Moisture', 'Pounds_Needed_Soil_Moisture',
       'Latitude_of_Field_Corner_#1 (lower left)',
       'Longitude_of_Field_Corner_#1 (lower left)',
       'Latitude_of_Field_Corner_#2 (lower right)',
       'Longitude_of_Field_Corner_#2 (lower right)',
       'Latitude_of_Field_Corner_#3 (upper right)',
       'Longitude_of_Field_C

In [ ]:
#Calculate the average latitude of the four field corners and store it in a new 'lat' column
df_metadata["lat"] = df_metadata[['Latitude_of_Field_Corner_#1 (lower left)', 'Latitude_of_Field_Corner_#2 (lower right)',
             'Latitude_of_Field_Corner_#3 (upper right)','Latitude_of_Field_Corner_#4 (upper left)',]].mean(axis=1)

#Calculate the average longitude of the four field corners and store it in a new 'lon' column
df_metadata["lon"] = df_metadata[['Longitude_of_Field_Corner_#1 (lower left)', 'Longitude_of_Field_Corner_#2 (lower right)',
            'Longitude_of_Field_Corner_#3 (upper right)','Longitude_of_Field_Corner_#4 (upper left)',]].mean(axis=1)


In [ ]:
import re

comments = df_metadata[['Issue/comment_#1', 'Issue/comment_#2', 'Issue/comment_#3',
                     'Issue/comment_#4', 'Issue/comment_#5', 'Issue/comment_#6',
                     'Issue/comment_#7', 'Issue/comment_#8']].copy()

def replace_keywords(text):
    """This function checks whether a given string matches any words defined in a predefined dictionary.

  Function will:
  If there is a match, the string is either replaced with a simplified category label (e.g., 'water', 'animal') or
  If the match is from the 'keywords' category, the actual matching word is retained.
  All non-matching strings and missing values are replaced with 'None' ."""

    # Define keywords and their categories
    keyword_categories = {
        "keywords": ["weed", "lodg", "feed", "stunt", "germination", "stand", "emergence", "rot"],
        "wind": ["wind", "gooseneck", "goose neck", "greensnap", "hail"],
        "weed" :["morning glory", "pigweed", "watchdog", "herbicide"],
        "water": ["drought", "dry", "flood", "rain", "moisture", "water", "underwater", "irrigation", "irrigated"],
        "equip": ["break", "battery", "jam", "disk", "cultivator", "weather", 
                  "sensor", "wire", "additional weather", "tractor", "roller", 
                  "plugged", "sprayer", "planter", "combine", "crew", "mistake", "miscommunication"],
        "animal": ["raccoon", "goose", "deer", "rodents", "hog"]
    }
    
    if isinstance(text, str):  
        text_lower = text.lower() 
        
        # Match each category with its radicals
        for category, radicals in keyword_categories.items():
            for radical in radicals:
                if re.search(rf"\b{radical}", text_lower): 
                    if category == "keywords":
                        return radical  
                    elif category == "wind":
                        return "wind damage"
                    elif category == "water":
                        return "water"
                    elif category == "weed":
                        return "weed"
                    elif category == "equip":
                        return "equipment"
                    elif category == "animal":
                        return "animal"
    return None  

In [ ]:
#apply function
for col in comments.columns :
    print(f"Processing column: {col}")
    comments[col] = comments[col].apply(replace_keywords) 

Processing column: Issue/comment_#1
Processing column: Issue/comment_#2
Processing column: Issue/comment_#3
Processing column: Issue/comment_#4
Processing column: Issue/comment_#5
Processing column: Issue/comment_#6
Processing column: Issue/comment_#7
Processing column: Issue/comment_#8


In [ ]:
unique_values = comments.stack().unique()
print(len(unique_values))
print(unique_values)

12
['weed' 'feed' 'lodg' 'equipment' 'stunt' 'water' 'stand' 'rot' 'animal'
 'germination' 'wind damage' 'emergence']


In [ ]:
# Multiple comments across columns are combined
df_metadata['Issues'] = comments.apply(lambda row: ', '.join(filter(None, row)), axis=1)

In [ ]:
# Extract key raw and processed management features
df_metadata_extracted = df_metadata[['Year', 'Env', 'Experiment_Code', 'Treatment',
       'Soil_Taxonomic_ID and horizon description, if known',
       'Previous_Crop', 'Pre-plant_tillage_method(s)',
       'In-season_tillage_method(s)', 'Issues', 'lon', 'lat',
       'Type_of_planter (fluted cone; belt cone; air planter)', 'Irrigated']]



In [ ]:
df_metadata_extracted.head(5)

,Year,Env,Experiment_Code,Treatment,"Soil_Taxonomic_ID and horizon description, if known",Previous_Crop,Pre-plant_tillage_method(s),In-season_tillage_method(s),Issues,lon,lat,Type_of_planter (fluted cone; belt cone; air planter),Irrigated
0,2014,DEH1_2014,DEH1,NaN,NaN,soybean,Conventional,NaN,NaN,NaN,NaN,Air planter,NaN
1,2014,GAH1_2014,GAH1,NaN,NaN,cotton,conventional,NaN,NaN,NaN,NaN,fluted cone,NaN
2,2014,IAH1a_2014,IAH1,NaN,NaN,soybean,field cultivator,NaN,NaN,NaN,NaN,Air planter,NaN
3,2014,IAH1b_2014,IAH1,NaN,NaN,soybean,field cultivator,NaN,NaN,NaN,NaN,Air planter,NaN
4,2014,IAH1c_2014,IAH1,NaN,NaN,soybean,field cultivator,NaN,NaN,NaN,NaN,Air planter,NaN


In [ ]:
missing_values = df_metadata_extracted.isnull().sum()
print(f"Missing values per column:\n{missing_values}")

Missing values per column:
Year                                                       0
Env                                                        0
Experiment_Code                                            0
Treatment                                                 56
Soil_Taxonomic_ID and horizon description, if known      195
Previous_Crop                                             34
Pre-plant_tillage_method(s)                               57
In-season_tillage_method(s)                              186
Issues                                                   178
lon                                                       52
lat                                                       52
Type_of_planter (fluted cone; belt cone; air planter)     25
Irrigated                                                222
dtype: int64


In [ ]:
print(df_metadata_extracted['Treatment'].value_counts(dropna=False))

Treatment
Standard                        192
NaN                              56
Drought                           8
Disease trial                     4
Dry Land                          2
Late Planting                     2
Early Planting                    1
Irrigated                         1
Late Stressed                     1
Dryland                           1
Standard - Irrigated Optimal      1
Dryland optimal                   1
Late Planted Irrigated            1
Late planting                     1
Name: count, dtype: int64


In [ ]:
df_metadata_extracted['Treatment'] = df_metadata_extracted['Treatment'].replace('Late planting', 'Late Planting')
df_metadata_extracted['Treatment'] = df_metadata_extracted['Treatment'].replace('Dryland', 'Dry Land')

In [ ]:
print(df_metadata_extracted['Previous_Crop'].value_counts(dropna=False))

Previous_Crop
soybean                                                                                                                               159
NaN                                                                                                                                    34
corn                                                                                                                                   30
wheat                                                                                                                                  16
peanut                                                                                                                                  6
Winter wheat                                                                                                                            4
sorghum                                                                                                                                 3
Sorghum             

In [ ]:
df_metadata_extracted['Previous_Crop'] = df_metadata_extracted['Previous_Crop'].replace('soybean', 'Soybeans')
df_metadata_extracted['Previous_Crop'] = df_metadata_extracted['Previous_Crop'].replace('sorghum', 'Sorghum')
df_metadata_extracted['Previous_Crop'] = df_metadata_extracted['Previous_Crop'].replace('Winter wheat', 'Winter Wheat')

df_metadata_extracted['Previous_Crop'] = df_metadata_extracted['Previous_Crop'].replace('wheat/double crop soybean', 'wheat/soybean')
df_metadata_extracted['Previous_Crop'] = df_metadata_extracted['Previous_Crop'].replace('wheat and Double Crop soybean', 'wheat/soybean')

In [ ]:
df_metadata_extracted['Pre-plant_tillage_method(s)'].unique().tolist()

['Conventional',
 'conventional',
 'field cultivator',
 'No-till',
 'Chisel plow and field cultivator',
 'Fall chisel plow and spring field cultivate',
 'chisel plow in fall; field cultivated in spring',
 'Disc in previous fall',
 'In the Spring the land was cut with a disk, then ripped with a chisel plow to a depth of 8-10”. It was then cut again and we applied 300#/acre of 10-0-30-12%S. Next we used a field cultivator with rolling baskets to incorporate the fertilizer. The land was bedded just prior to planting.',
 'chisel',
 'no-till',
 'Field J was fall moldboard plow;  Then disked this spring and field cultivated before planting.',
 'The field was minium tilled.  The field was disked then cultipacked then Cultimulched then planted',
 'Fall Chisel Plow; Spring Cultivate',
 'min-till',
 nan,
 'Fall Chisel',
 'Field cultivator',
 'Field cultivate',
 'fall chisel plow, spring field cultivator',
 'disc, conventional, followed by bedding',
 'No-Till',
 'No Till',
 'Chisel plowed 5/4/15 

In [ ]:
df_metadata_extracted['Pre-plant_tillage_method(s)'] = df_metadata_extracted['Pre-plant_tillage_method(s)'].str.lower()
df_metadata_extracted['Pre-plant_tillage_method(s)'] = df_metadata_extracted['Pre-plant_tillage_method(s)'].replace('disc', 'disk')
#minimal editing of values.
#for example, Manual weeding could be hand weed but went unaltered.


In [ ]:
print(df_metadata_extracted['In-season_tillage_method(s)'].value_counts(dropna=False))

In-season_tillage_method(s)
NaN                                                         186
none                                                         44
Cultivator                                                   10
Hand hoeing                                                   4
CaseIH VT 360 vertical tillage tool gone over 2X on 5/30      3
Cultivator tilling                                            3
hand weed                                                     2
disc                                                          2
Disked on 05/17/18 AM, Rolling Harrow on 05/17/18 PM          2
cultivate                                                     2
disk                                                          1
cultivation                                                   1
plow                                                          1
Disked, chisel plow & final disking                           1
Disced and cultimulched                                       1
Field Cultiv

In [ ]:
df_metadata_extracted['In-season_tillage_method(s)'] = df_metadata_extracted['In-season_tillage_method(s)'].str.lower()
df_metadata_extracted['In-season_tillage_method(s)'] = df_metadata_extracted['In-season_tillage_method(s)'].replace('disc', 'disk')


In [ ]:
print(df_metadata_extracted['Type_of_planter (fluted cone; belt cone; air planter)'].value_counts(dropna=False))

Type_of_planter (fluted cone; belt cone; air planter)
air planter                                       151
fluted cone                                        36
belt cone                                          26
NaN                                                25
Air planter                                         7
SRES Air                                            3
vacuum precision planter                            3
air                                                 2
Almaco TP2                                          2
Fluted cone                                         2
4 row almaco GPS Drop precision vacuum planter      2
Almaco Seed Pro 360                                 2
Almaco 4-row air planter                            2
Fluted Cone                                         1
air (Seedpro)                                       1
John Deere 7300                                     1
vacuum planter                                      1
Vacuum                      

In [ ]:
df_metadata_extracted['Type_of_planter (fluted cone; belt cone; air planter)'] = df_metadata_extracted['Type_of_planter (fluted cone; belt cone; air planter)'].str.lower()
df_metadata_extracted['Type_of_planter (fluted cone; belt cone; air planter)'] = df_metadata_extracted['Type_of_planter (fluted cone; belt cone; air planter)'].replace('vacum', 'vacuum')
df_metadata_extracted['Type_of_planter (fluted cone; belt cone; air planter)'] = df_metadata_extracted['Type_of_planter (fluted cone; belt cone; air planter)'].replace('flute cone', 'fluted cone')
df_metadata_extracted['Type_of_planter (fluted cone; belt cone; air planter)'] = df_metadata_extracted['Type_of_planter (fluted cone; belt cone; air planter)'].replace('air', 'air planter')

In [ ]:
print(df_metadata_extracted['Irrigated'].value_counts(dropna=False))

Irrigated
NaN    222
no      31
yes     19
Name: count, dtype: int64


In [ ]:
df_metadata_filled = df_metadata_extracted.fillna('n/a')

In [ ]:
df_metadata_filled.head(5)

,Year,Env,Experiment_Code,Treatment,"Soil_Taxonomic_ID and horizon description, if known",Previous_Crop,Pre-plant_tillage_method(s),In-season_tillage_method(s),Issues,lon,lat,Type_of_planter (fluted cone; belt cone; air planter),Irrigated
0,2014,DEH1_2014,DEH1,n/a,n/a,Soybeans,conventional,n/a,n/a,n/a,n/a,air planter,n/a
1,2014,GAH1_2014,GAH1,n/a,n/a,cotton,conventional,n/a,n/a,n/a,n/a,fluted cone,n/a
2,2014,IAH1a_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,n/a,n/a,air planter,n/a
3,2014,IAH1b_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,n/a,n/a,air planter,n/a
4,2014,IAH1c_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,n/a,n/a,air planter,n/a


#### Find substitute environment to fill in missing lat and lon values

Three coordinate matching options:
- Year-locations grown in same city was used as subsitutes for year-locations with missing lat and lon measurements <br>

For year-locations with no substitute found: 
-  An year-location in same region was used as substitute (missing city info) <br>
-  Lat/lons were imputed manually after searching (if city was known)

In [ ]:
df_missing_lat_lon = df_metadata_filled[(df_metadata_filled['lat'] == 'n/a') & (df_metadata_filled['lon'] == 'n/a')]
print(len(df_missing_lat_lon))

missing_env_list = df_missing_lat_lon['Env'].tolist()

52


In [ ]:
missing_env_list

['DEH1_2014',
 'GAH1_2014',
 'IAH1a_2014',
 'IAH1b_2014',
 'IAH1c_2014',
 'IAH2_2014',
 'IAH3_2014',
 'IAH4_2014',
 'ILH1_2014',
 'INH1_2014',
 'MNH1_2014',
 'MOH1_2014',
 'MOH2_2014',
 'NCH1_2014',
 'NEH1_2014',
 'NEH2_2014',
 'NEH3_2014',
 'NYH1_2014',
 'NYH2_2014',
 'ONH1_2014',
 'ONH2_2014',
 'TXH1_2014',
 'TXH2_2014',
 'WIH1_2014',
 'NYH1_2015',
 'TXH2_2015',
 'IAH1_2016',
 'IAH2_2016',
 'IAH3_2016',
 'IAH4_2016',
 'NEH1_2016',
 'NEH4_2016',
 'NYH1_2016',
 'TXH2_2016',
 'ILH1_2017',
 'INH1_2017',
 'MNH1_2017',
 'TXH1-Dry_2017',
 'TXH1-Early_2017',
 'TXH1-Late_2017',
 'TXH2_2017',
 'IAH1_2018',
 'IAH2_2018',
 'IAH3_2018',
 'IAH4_2018',
 'TXH2_2018',
 'NEH2_2019',
 'TXH4_2019',
 'NEH2_2020',
 'NEH3_2020',
 'ILH1_2021',
 'NYS1_2021']

In [ ]:
# Upload original metadata file with City info
df_metadata_org = pd.read_csv('2_Training_Meta_Data_2014_2023.csv')

In [ ]:
df_metadata_org.head(5)

,Year,Env,Experiment_Code,Treatment,City,Farm,Field,Trial_ID (Assigned by collaborator for internal reference),"Soil_Taxonomic_ID and horizon description, if known","Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)",...,Cardinal_Heading_Pass_1,Irrigated,Issue/comment_#1,Issue/comment_#2,Issue/comment_#3,Issue/comment_#4,Issue/comment_#5,Issue/comment_#6,Issue/comment_#7,Issue/comment_#8
0,2014,DEH1_2014,DEH1,NaN,Georgetown,Elbert N. & Ann V. Carvel Research & Education...,27AB,NaN,NaN,9079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,GAH1_2014,GAH1,NaN,Tifton,USDA - Bellflower experimental farm,18,NaN,NaN,8427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,IAH1a_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN
3,2014,IAH1b_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN
4,2014,IAH1c_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN


In [ ]:
#create a dataframe with Env, City columns of instances WITHOUT lat and lon measurements
df_missing_env_cities = df_metadata_org[df_metadata_org['Env'].isin(missing_env_list)][['Env', 'City']]

In [ ]:
df_missing_env_cities.head(5)

,Env,City
0,DEH1_2014,Georgetown
1,GAH1_2014,Tifton
2,IAH1a_2014,Ames
3,IAH1b_2014,Ames
4,IAH1c_2014,Ames


In [ ]:
df_sub_pool = df_metadata_org[
    df_metadata_org['Latitude_of_Field_Corner_#1 (lower left)'].notna() & df_metadata_org['Longitude_of_Field_Corner_#1 (lower left)'].notna()][['Env', 'City']]

In [ ]:
len(df_sub_pool), len(df_missing_env_cities)

(220, 52)

In [ ]:
#find substitute envs from sub pool 
df_substitute_envs = df_missing_env_cities.merge(
    df_sub_pool,
    on='City',
    how='left',  #ensures envs without substitutes are retained
    suffixes=('_missing', '_substitute')
)

In [ ]:
df_substitute_envs.head(5)
#mulitple subs from across different years found 

,Env_missing,City,Env_substitute
0,DEH1_2014,Georgetown,DEH1_2015
1,DEH1_2014,Georgetown,DEH1_2016
2,DEH1_2014,Georgetown,DEH1_2017
3,DEH1_2014,Georgetown,DEH1_2018
4,DEH1_2014,Georgetown,DEH1_2019


In [ ]:
#select only first substitute env for those with many substitutes.
df_substitute_envs = df_substitute_envs.groupby('Env_missing').first().reset_index()

In [ ]:
df_substitute_envs.head(5)

,Env_missing,City,Env_substitute
0,DEH1_2014,Georgetown,DEH1_2015
1,GAH1_2014,Tifton,GAH1_2015
2,IAH1_2016,Crawfordsville,IAH4_2015
3,IAH1_2018,Crawfordsville,IAH4_2015
4,IAH1a_2014,Ames,IAH1_2015


In [ ]:
#Still missing  environments(no city level matches)
print('missing cities/subs:')
df_substitute_envs.isna().sum()


missing cities/subs:


Env_missing       0
City              5
Env_substitute    8
dtype: int64

In [ ]:
df_still_missing = df_substitute_envs[
    df_substitute_envs['City'].isna() | df_substitute_envs['Env_substitute'].isna()]

In [ ]:
df_still_missing
# No city info for 5 year-locations (envs) (location - level matching)
# City info for 3 year-locations (envs) (manual search - lat/lon)

,Env_missing,City,Env_substitute
17,ILH1_2017,None,None
20,INH1_2017,None,None
45,TXH2_2014,Halfway,None
46,TXH2_2015,None,None
47,TXH2_2016,None,None
48,TXH2_2017,None,None
49,TXH2_2018,Lubbock,None
50,TXH4_2019,Lubbock,None


In [ ]:
# Location level matching

df_metadata_org[df_metadata_org['Experiment_Code'] == 'ILH1']
#looks like after 2016 ILHI has been in Champaign
#therefore let ILH1_2017 substitute env be ILH1_2018 

,Year,Env,Experiment_Code,Treatment,City,Farm,Field,Trial_ID (Assigned by collaborator for internal reference),"Soil_Taxonomic_ID and horizon description, if known","Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)",...,Cardinal_Heading_Pass_1,Irrigated,Issue/comment_#1,Issue/comment_#2,Issue/comment_#3,Issue/comment_#4,Issue/comment_#5,Issue/comment_#6,Issue/comment_#7,Issue/comment_#8
8,2014,ILH1_2014,ILH1,NaN,Urbana,Maxwell Farms,MF500,NaN,NaN,8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,2015,ILH1_2015,ILH1,NaN,Urbana,NaN,NaN,NaN,NaN,8653,...,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,2016,ILH1_2016,ILH1,Standard,Champaign,Maxwell Farm,MF-500,NaN,Flanagan Silt Loam (154A),8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,2017,ILH1_2017,ILH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,2018,ILH1_2018,ILH1,Standard,Champaign,Maxwell Farm,MF200,ILH1,Flanagan Silt Loam,8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,2019,ILH1_2019,ILH1,Standard,Champaign,Maxwell Farm,MF1000,ILH1,Flanagan Silt Loam,8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,2021,ILH1_2021,ILH1,Standard,Champaign,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,2022,ILH1_2022,ILH1,Standard,Champaign,South Farms,AnSci200,NaN,NaN,8653,...,NaN,no,Link to additional weather source available on...,"In general, dry and hot summer.",NaN,NaN,NaN,NaN,NaN,NaN
253,2023,ILH1_2023,ILH1,Standard,Champaign,South Farms,S700,NaN,"Drummer silty clay loam, 0 to 2 percent slopes...",8653,...,NaN,no,Link to additional weather source available on...,"In general, dry and hot summer.",NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_metadata_org[df_metadata_org['Experiment_Code'] == 'INH1']
#looks like INH1 is always in West Lafayette and farm is ACRE
#therefore let INH1_2017 sub be INH1_2018

,Year,Env,Experiment_Code,Treatment,City,Farm,Field,Trial_ID (Assigned by collaborator for internal reference),"Soil_Taxonomic_ID and horizon description, if known","Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)",...,Cardinal_Heading_Pass_1,Irrigated,Issue/comment_#1,Issue/comment_#2,Issue/comment_#3,Issue/comment_#4,Issue/comment_#5,Issue/comment_#6,Issue/comment_#7,Issue/comment_#8
9,2014,INH1_2014,INH1,NaN,West Lafayette,Purdue ACRE,97/98,NaN,NaN,8657,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,2015,INH1_2015,INH1,NaN,West Lafayette,NaN,NaN,NaN,NaN,8657,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,2016,INH1_2016,INH1,NaN,West Lafayette,Purdue ACRE,54 North,NaN,NaN,8657,...,0,NaN,"At planting, a row unit plugged up with a clod...",July 14. Noticed that Moisture sensor wire was...,NaN,NaN,NaN,NaN,NaN,NaN
89,2017,INH1_2017,INH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,2018,INH1_2018,INH1,Standard,West Lafayette,Purdue ACRE,54NN,GxE,RcA-Raub-Brenton Complex,8657,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,2019,INH1_2019,INH1,Standard,West Lafayette,Purdue ACRE,42,INH1,Raub-Brenton complex,8657,...,0,NaN,Bulk Density sensor started actigng up during ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2020,INH1_2020,INH1,Standard,West Lafayette,Purdue ACRE,54N,G2F,Raub-Brenton complex,8657,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,2021,INH1_2021,INH1,Standard,West Lafayette,Purdue ACRE,42,GxE,Chalmers Silty Clay Loam,8657,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,2022,INH1_2022,INH1,Standard,West Lafayette,ACRE,78,GxE,Raub/Brenton Complex,8657,...,0,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,2023,INH1_2023,INH1,Standard,West Lafayette,ACRE,42,GxE,Chalmers Silty Clay Loam,8657,...,0,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_metadata_org[df_metadata_org['Experiment_Code'] == 'TXH2']
#same problem as weather subs: could be Halfway, Lubbock or College Station (no consistency during 2015-2017 period)
#could either assume mode: College station (mode) or assume previous year: Halfway
#lets remain consistent with weather sub and assume College Station
#subs for TXH2_2015,2016,2017 is TXH2_2019

,Year,Env,Experiment_Code,Treatment,City,Farm,Field,Trial_ID (Assigned by collaborator for internal reference),"Soil_Taxonomic_ID and horizon description, if known","Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)",...,Cardinal_Heading_Pass_1,Irrigated,Issue/comment_#1,Issue/comment_#2,Issue/comment_#3,Issue/comment_#4,Issue/comment_#5,Issue/comment_#6,Issue/comment_#7,Issue/comment_#8
22,2014,TXH2_2014,TXH2,NaN,Halfway,Halfway,pivot,NaN,NaN,8630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,2015,TXH2_2015,TXH2,NaN,NaN,NaN,NaN,NaN,NaN,8630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,2016,TXH2_2016,TXH2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,2017,TXH2_2017,TXH2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,2018,TXH2_2018,TXH2,Standard,Lubbock,NaN,NaN,NaN,NaN,8630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,2019,TXH2_2019,TXH2,Standard,College Station,NaN,NaN,NaN,NaN,14790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,2020,TXH2_2020,TXH2,Dry Land,College Station,NaN,NaN,NaN,NaN,14790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,2021,TXH2_2021,TXH2,Dryland optimal,College Station,Texas A&M Research Farm,302,G2F2,Belk Clay Loam (BaA) / Ships Clay Loam (ShA),14790,...,310,NaN,There was a substantial morning glory infestation,Coordinates approximate from Google Maps,NaN,NaN,NaN,NaN,NaN,NaN
240,2022,TXH2_2022,TXH2,Drought,College Station,Texas A&M Research Farm,302,TXH,NaN,14790,...,312,no,The post harvest herbicide spray was to get ah...,Plots 325-500 where irrigated due to a busted ...,NaN,NaN,NaN,NaN,NaN,NaN
267,2023,TXH2_2023,TXH2,Drought,College Station,Texas Research Farm,224,TXH2,NaN,NaN,...,45,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Dict within a dict
subs_round_2 = {
    'ILH1_2017': {'City': 'Champaign', 'Env_substitute': 'ILH1_2018'},
    'INH1_2017': {'City': 'West Lafayette', 'Env_substitute': 'INH1_2018'},
    'TXH2_2015': {'City': 'College Station', 'Env_substitute': 'TXH2_2019'},
    'TXH2_2016': {'City': 'College Station', 'Env_substitute': 'TXH2_2019'},
    'TXH2_2017': {'City': 'College Station', 'Env_substitute': 'TXH2_2019'},
}

#Add subsitite env based on location level matching
for env, info in subs_round_2.items():
    df_substitute_envs.loc[df_substitute_envs['Env_missing'] == env, 'City'] = info['City'] 
    df_substitute_envs.loc[df_substitute_envs['Env_missing'] == env, 'Env_substitute'] = info['Env_substitute'] 


In [ ]:
df_substitute_envs.isna().sum()

Env_missing       0
City              0
Env_substitute    3
dtype: int64

In [ ]:
df_substitute_envs_v2 = df_substitute_envs.dropna()
#remove TXH2_2014,2018,TXH4_2019 need to be added after internet search

In [ ]:
# Manual Search for remaining 3 missing year-locations

df_metadata_filled.tail(5)
#lat  -90 to 90 (south-north) 
#long -180 to 180 (west-east) 

#Lubbock coordinates: lat= 33.5779, lon= -101.8552 for TXH2_2018, TXH4_2019 
#Halfway coordinates: lat=34.1881, lon=-101.9524 for TXH2_201411


,Year,Env,Experiment_Code,Treatment,"Soil_Taxonomic_ID and horizon description, if known",Previous_Crop,Pre-plant_tillage_method(s),In-season_tillage_method(s),Issues,lon,lat,Type_of_planter (fluted cone; belt cone; air planter),Irrigated
267,2023,TXH2_2023,TXH2,Drought,n/a,Sorghum,discing,cultivator,n/a,-96.43165871,30.55034681,belt cone,no
268,2023,TXH3_2023,TXH3,Late Planting,n/a,Sorghum,disicing,cultivator,n/a,-96.4310292925,30.549647405,belt cone,yes
269,2023,WIH1_2023,WIH1,Standard,"PoA, TrB",Soybeans,field cultivator,n/a,n/a,-89.5297795,43.0554021125,air planter,no
270,2023,WIH2_2023,WIH2,Standard,"PnA, PnB",Soybeans,field cultivator,n/a,equipment,-89.3841457,43.304242975,air planter,no
271,2023,WIH3_2023,WIH3,Standard,Sandy,n/a,"disc, dynadrive",n/a,equipment,-89.54400899999999,44.115644625,air planter,yes


In [ ]:
manual_lat_lon_info = {
    'TXH2_2018': {'Lat': 33.5779, 'Lon': -101.8552},  # Lubbock
    'TXH4_2019': {'Lat': 33.5779, 'Lon': -101.8552},  # Lubbock
    'TXH2_2014': {'Lat': 34.1881, 'Lon': -101.9524},  # Halfway
}

# add manual lat/lon coordinates straight to metadata df
for env, coords in manual_lat_lon_info.items():
    df_metadata_filled.loc[df_metadata_filled['Env'] == env, 'lat'] = coords['Lat']
    df_metadata_filled.loc[df_metadata_filled['Env'] == env, 'lon'] = coords['Lon']

In [ ]:
df_substitute_envs_v2.head(5)

,Env_missing,City,Env_substitute
0,DEH1_2014,Georgetown,DEH1_2015
1,GAH1_2014,Tifton,GAH1_2015
2,IAH1_2016,Crawfordsville,IAH4_2015
3,IAH1_2018,Crawfordsville,IAH4_2015
4,IAH1a_2014,Ames,IAH1_2015


In [ ]:
# Add location matching lat-lon missing values to 'df_metadata_filled'

for _, row in df_substitute_envs_v2.iterrows():
    env_missing = row['Env_missing'] 
    env_substitute = row['Env_substitute'] 
    
    lat_substitute = df_metadata_filled.loc[df_metadata_filled['Env'] == env_substitute, 'lat'].values 
    lon_substitute = df_metadata_filled.loc[df_metadata_filled['Env'] == env_substitute, 'lon'].values 

    df_metadata_filled.loc[df_metadata_filled['Env'] == env_missing, 'lat'] = lat_substitute 
    df_metadata_filled.loc[df_metadata_filled['Env'] == env_missing, 'lon'] = lon_substitute 

In [ ]:
df_metadata_filled.head(5)
#lat and lon have been imputed (see first 5 lon and lat columns)

,Year,Env,Experiment_Code,Treatment,"Soil_Taxonomic_ID and horizon description, if known",Previous_Crop,Pre-plant_tillage_method(s),In-season_tillage_method(s),Issues,lon,lat,Type_of_planter (fluted cone; belt cone; air planter),Irrigated
0,2014,DEH1_2014,DEH1,n/a,n/a,Soybeans,conventional,n/a,n/a,-75.4656925,38.62935675,air planter,n/a
1,2014,GAH1_2014,GAH1,n/a,n/a,cotton,conventional,n/a,n/a,-83.555095,31.505771,fluted cone,n/a
2,2014,IAH1a_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,-93.693369725,41.99809519750001,air planter,n/a
3,2014,IAH1b_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,-93.693369725,41.99809519750001,air planter,n/a
4,2014,IAH1c_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,-93.693369725,41.99809519750001,air planter,n/a


In [ ]:
df_metadata_filled['lat'].str.contains('n/a').sum(),df_metadata_filled['lon'].str.contains('n/a').sum()
#all lats and lons imputed (city-level, location wise and manual search)

(0, 0)

#### Label Encode and/or Normalise all metadatafeatures

In [ ]:
df_metadata_filled.dtypes
#only label encode object cols

Year                                                       int64
Env                                                       object
Experiment_Code                                           object
Treatment                                                 object
Soil_Taxonomic_ID and horizon description, if known       object
Previous_Crop                                             object
Pre-plant_tillage_method(s)                               object
In-season_tillage_method(s)                               object
Issues                                                    object
lon                                                      float64
lat                                                      float64
Type_of_planter (fluted cone; belt cone; air planter)     object
Irrigated                                                 object
dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encode_features(df: pd.DataFrame, exclude_cols: list = None):
    """
    Label encodes all object columns in a dataframe, excluding any specified in exclude_cols.
    Deletes the original columns after encoding.
        
    Returns:
        pd.DataFrame: DataFrame with label-encoded columns replacing orginal columns
    """
    df = df.copy()
    le = LabelEncoder()
    
    if exclude_cols is None:
        exclude_cols = []
    
    for col in df.columns:
        if col not in exclude_cols and df[col].dtype == 'object': #only proceed if cols are objects
            df[col + '_Label'] = le.fit_transform(df[col].astype(str)) #converts values in object cols to strings
            df.drop(columns=col, inplace=True)
    
    return df


In [ ]:
df_metadata_encoded = label_encode_features(df_metadata_filled, exclude_cols=['Env','Year','lat', 'lon']) 

In [ ]:
df_metadata_encoded.head(5)

,Year,Env,lon,lat,Experiment_Code_Label,Treatment_Label,"Soil_Taxonomic_ID and horizon description, if known_Label",Previous_Crop_Label,Pre-plant_tillage_method(s)_Label,In-season_tillage_method(s)_Label,Issues_Label,Type_of_planter (fluted cone; belt cone; air planter)_Label,Irrigated_Label
0,2014,DEH1_2014,-75.465693,38.629357,3,11,48,6,23,18,24,2,0
1,2014,GAH1_2014,-83.555095,31.505771,4,11,48,10,23,18,24,7,0
2,2014,IAH1a_2014,-93.693370,41.998095,7,11,48,6,69,18,24,2,0
3,2014,IAH1b_2014,-93.693370,41.998095,7,11,48,6,69,18,24,2,0
4,2014,IAH1c_2014,-93.693370,41.998095,7,11,48,6,69,18,24,2,0


In [46]:
from sklearn.preprocessing import MinMaxScaler

def normalize_features(df: pd.DataFrame, exclude_cols: list = None):
    """
    Normalizes all numeric columns in a dataframe using MinMaxScaler, excluding any specified in exclude_cols. 
    Deletes the original columns after normalization.
    
    Returns:
        pd.DataFrame: DataFrame with normalized columns replacing original columns
    """
    df = df.copy()
    scaler = MinMaxScaler()
    
    if exclude_cols is None:
        exclude_cols = []

    for col in df.select_dtypes(include=['number']).columns: #only proceed if features are numeric regardless of whether it is int or float
        if col not in exclude_cols:
            df[col + '_Normalized'] = scaler.fit_transform(df[[col]])
            df.drop(columns=col, inplace=True)

    return df


In [ ]:
df_metadata_encoded_normalised = normalize_features(df_metadata_encoded, exclude_cols=['Env']) 

In [ ]:
df_metadata_encoded_normalised.head(5)

,Env,Year_Normalized,lon_Normalized,lat_Normalized,Experiment_Code_Label_Normalized,Treatment_Label_Normalized,"Soil_Taxonomic_ID and horizon description, if known_Label_Normalized",Previous_Crop_Label_Normalized,Pre-plant_tillage_method(s)_Label_Normalized,In-season_tillage_method(s)_Label_Normalized,Issues_Label_Normalized,Type_of_planter (fluted cone; belt cone; air planter)_Label_Normalized,Irrigated_Label_Normalized
0,DEH1_2014,0.0,0.256959,0.385771,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0
1,GAH1_2014,0.0,0.186578,0.046148,0.093023,1.0,0.979592,0.588235,0.247312,0.857143,0.533333,0.466667,0.0
2,IAH1a_2014,0.0,0.098371,0.546379,0.162791,1.0,0.979592,0.352941,0.741935,0.857143,0.533333,0.133333,0.0
3,IAH1b_2014,0.0,0.098371,0.546379,0.162791,1.0,0.979592,0.352941,0.741935,0.857143,0.533333,0.133333,0.0
4,IAH1c_2014,0.0,0.098371,0.546379,0.162791,1.0,0.979592,0.352941,0.741935,0.857143,0.533333,0.133333,0.0
